In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.5, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class nobel prized poet. You only create poems about programming languages. You are famous for using easy expressions in your poetry. Your poetry should be in english, and length should be min 100 words and max 500 words."),
    ("human", "Write a poem about {language}")
])

poet_chain = poet_prompt | chat

In [9]:
analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional poem analysist. You summarize and analyze poetry with easy expressions for middle school students. Length of your analysis should be max 350 words."),
    ("human", "{poem}")
])

analysis_chain = analysis_prompt | chat

final_chain = {"poem":poet_chain} | analysis_chain

final_chain.invoke({
    "language" : "typescript"
})

In the world of code, a language stands tall,
TypeScript, beloved by one and all.
With types to guide and errors to catch,
It makes programming a smoother match.

No more guessing, no more fear,
TypeScript whispers in your ear.
"Is that variable a string or a number?
Let me help you, I won't let you slumber."

Static typing, oh so divine,
Makes your code robust and fine.
No more runtime errors to dread,
With TypeScript, your worries are shed.

Classes, interfaces, generics galore,
TypeScript offers so much more.
A superset of JavaScript, it truly shines,
In the land of programming, it defines.

So raise your hands, give a cheer,
For TypeScript, the language we hold dear.
With its strong typing and modern flair,
It's a programmer's dream, beyond compare.This poem celebrates TypeScript, a programming language highly regarded in the coding world. The poet praises TypeScript for its feature of "types" that help programmers identify and prevent errors in their code, making the coding proces

AIMessage(content='This poem celebrates TypeScript, a programming language highly regarded in the coding world. The poet praises TypeScript for its feature of "types" that help programmers identify and prevent errors in their code, making the coding process smoother and less stressful.\n\nThe poet highlights how TypeScript eliminates the need for guesswork and fear in coding by providing clear guidance on variable types. The use of "static typing" is commended for making the code more robust and reducing the occurrence of runtime errors, offering a sense of relief to programmers.\n\nThe poem mentions various features of TypeScript such as classes, interfaces, and generics, showcasing the language\'s versatility and richness compared to its base language, JavaScript. TypeScript is portrayed as a superior choice in the realm of programming due to its modernity and precision.\n\nOverall, the poem encourages readers to appreciate and celebrate TypeScript for its strong typing system and co

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import example_selector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(temperature=0.5, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# placeholder이름을 key변수로 사용함. 
examples = [
    { "company":"Inditex",
    "reply":"""
    This is what I know:
    Country : Spain
    Industry : Fashion
    Address : La Coruna, Spain
    Found : 1985
    Total Sales : 35.9 billion euros (2023)
    Gross Profit : 20.8 billion euros (2023)
    """
    }, {
        "company":"Tesla",
    "reply":"""
    This is what I know:
    Country : USA
    Industry : Automobile
    Address : Austin, Texas, USA
    Found : 2003
    Total Sales : 71.6 billion dollars (2023) appx
    Gross Profit : 17.66 billion dollars (2023)
    """
    }
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    # 직접 example selector을 만들면, 그 example selector은 add_example 메소드를 필수적으로 필요로 한다.
    # Adds examples to your already existing examples...뭔 일을 하는 메소드인거지 대체
    def add_example(self, example):
        self.examples.append(example)
    # select examples method
    # 예시 리스트로부터 무작위로 한개의 예시를 골라 return함. return값은 list형식[]이어야 함.
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human:{company}\nAI:{reply}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human: what do you know about {company}",
    input_variables = ["company"],
)


prompt.format(company="LG")

'Human:Tesla\nAI:\n    This is what I know:\n    Country : USA\n    Industry : Automobile\n    Address : Austin, Texas, USA\n    Found : 2003\n    Total Sales : 71.6 billion dollars (2023) appx\n    Gross Profit : 17.66 billion dollars (2023)\n    \n\nHuman: what do you know about LG'

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(temperature=0.3, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

prompt = load_prompt("./prompt.json")

prompt.format(country="Sweden")

'What is the capital of Sweden'

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(temperature=0.3, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

intro = PromptTemplate.from_template(
    """
    you are a role playing assistant. and you are impersonating a {character}
    """
)

# 위에서 placeholder로 설정한 캐릭터가 어떻게 말하는지 예시 제공
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:
    Human: {example_question}
    You: {example_answer}
    """
)
# AI asistant가 우리의 텍스트를 완성해줄 예시
start = PromptTemplate.from_template(
    """
    Start now!
    Human: {question}
    You: 
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
    {example}
    {start}
    """
)

prompts = [
    ("intro", intro), ("example",example),("start",start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

full_prompt.format(
    character="cinderella", example_question="what are you doing right now?",
    example_answer="Oh, I'm cleaning the house! Look at me sweep!",
    question="What is your favorite food?"
)

"\n    \n    you are a role playing assistant. and you are impersonating a cinderella\n    \n    \n    This is an example of how you talk:\n    Human: what are you doing right now?\n    You: Oh, I'm cleaning the house! Look at me sweep!\n    \n    \n    Start now!\n    Human: What is your favorite food?\n    You: \n    \n    "

In [21]:
chain = full_prompt | chat

chain.invoke({
    "character":"cinderella", 
    "example_question":"what are you doing right now?",
    "example_answer":"Oh, I'm cleaning the house! Look at me sweep!",
    "question":"What is your favorite food?"
})

Oh, my favorite food is definitely pumpkin pie! It's so delicious and reminds me of the magical ball where I met my prince charming.

AIMessage(content="Oh, my favorite food is definitely pumpkin pie! It's so delicious and reminds me of the magical ball where I met my prince charming.", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-3fcc2049-213e-45d6-b238-11deeff459d4-0')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# LLM에서 받은 답변은 모두 메모리에 캐싱된다. 
set_llm_cache(InMemoryCache())

chat = ChatOpenAI(temperature=0.3, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])